<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/producto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Solución final al problema de generar edificio a lo largo de un camino***




# Introducción

###"*Caminante, no hay camino. Se hace camino al andar*"
Antonio Machado *1912*


---

En los últimos meses se ha avanzado mucho en lo correspondiente a generación de plantas de edificios y su distribución. El enfoque el éste trabajo es radicalmente distinto. Nosotros pretendemos ponernos en la piel del transeunte y vivir la experiencia de recorrer los edificios y las sensaciones que te generan. Para ello se toma como input una curva paramétrica de 3 dimensiones que representa el camino, la dividimos en trozos muy pequeños y realizamos las visuales con forma de nube de puntos en cada una y que de una intuición al arquitecto de por donde seguir en su diseño. Una visual en arquitectura como su propio nombre indica es la percepción en 360º desde una posición determinada de un edificio. La forma más eficiente es, usando un sistema de coordenadas esféricas tomar ángulos fijos y calcular la tercera coordenada, que es el radio. De esta forma obtenemos una esfera de distancias que puede ser simplificada en una matriz de distancas usando una proyección de Mercator. Dichas matrices son lo que el modelo genera. Después de obtener todas las visuales correspondientes se calcula la transformación inversa a la proyección de Mercator y se transporta cada una al punto indicado en la curva. Así se genera la nube de puntos de todo el camino através del edificio como si de un hormiguero se tratara.


#Librerías

---



In [0]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Conv2DTranspose, Reshape
from keras.models import Model ,Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

# Visualización y preprocesamiento de datos

---



In [42]:
input=np.random.random_sample((128,128))
print(input)

[[0.66872527 0.50201433 0.2610333  ... 0.71180088 0.48648238 0.3424924 ]
 [0.45952864 0.93984101 0.13826931 ... 0.98918115 0.1139043  0.39400716]
 [0.12761532 0.45475703 0.51444306 ... 0.49979997 0.56977931 0.12704229]
 ...
 [0.94348285 0.34247194 0.60873262 ... 0.76758931 0.69043367 0.82606595]
 [0.22651634 0.2118114  0.30115265 ... 0.13069027 0.67215384 0.02267914]
 [0.9158766  0.17884721 0.19119157 ... 0.54120307 0.64210222 0.25187294]]


# Implementación del sistema

---



### Optimizador ADAM

In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

## DCGAN generadora de la primera visual

---

Para obtener una visual nueva por donde empezar y totalmente independiente de cualquier edificio se usa una GAN (Generative Adversarial Network) que será entrenada con miles de visuales de edificios de arquitectos famosos como Le Corbusier. Una GAN consta de dos redes, el generador y el discriminador. La primera se encarga de generar visuales a partir de un vector de ruido aleatorio. La segunda calcula si la visual ha sido generada mediante el generador o es una visual de un edificio real. Las dos redes van mejorando simultaneamente en sus tareas hasta que el generador es capaz de crear visuales que parecen reales. En ese momento se desecha el discriminador y se usa solo el generador para crear la primera pieza de dominó que servirá para ir creando el resto según se avance por el camino.



### Generador

El generador toma como input un vector de 128 posiciones de ruido aleatorio y lo transforma en un output con forma de matriz ( , ). Para ello se construye en base a una red deconvolucional. La matriz de salida representa una proyección de Mercator de la visual en 3d. 




In [44]:
# input array 128
# output matriz (20,60)

def create_generator():

  generator=Sequential()

  generator.add(Dense(128 * 5 * 5, input_dim=128, name='dense'))
  generator.add(Reshape((5, 5, 128), name='reshape'))

  generator.add(Conv2DTranspose(64, (5,5), strides=(2,3), padding='same', name='conv2dt_1'))
  generator.add(Conv2DTranspose(1, (5,5), strides=(2,4), padding='same', name='conv2dt_2'))

  generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
  generator.name='generador'
  return generator
g = create_generator()
g.summary()

Model: "generador"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3200)              412800    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
conv2dt_1 (Conv2DTranspose)  (None, 10, 15, 64)        204864    
_________________________________________________________________
conv2dt_2 (Conv2DTranspose)  (None, 20, 60, 1)         1601      
Total params: 619,265
Trainable params: 619,265
Non-trainable params: 0
_________________________________________________________________


### Discriminador

El discriminador toma o una visual real o una creada por el generador y calcula si es real o generada. 

In [45]:
# input matriz (20,60)
# output valor entre 0 y 1
def create_discriminator():
  
    discriminator=Sequential()

    discriminator.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),input_shape=(20,60,1),name='conv2d_1'))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='maxpool_1'))
    
    discriminator.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),name='conv2d_2'))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='maxpool_2'))
    
    discriminator.add(Flatten())
    discriminator.add(Dense(units=1, activation='softmax'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    discriminator.name='discriminador'
    return discriminator
d = create_discriminator()
d.summary()

Model: "discriminador"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 56, 32)        832       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 16, 56, 32)        0         
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 15, 55, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 51, 32)        25632     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 11, 51, 32)        0         
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 10, 50, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 16000)           

### DCGAN

Se pone en común el trabajo del generador y del discriminador para que mejoren de forma conjunta.

In [46]:
def create_dcgan(discriminator, generator):

  discriminator.trainable=False
  gan_input = Input(shape=(128,))
  x = generator(gan_input)
  gan_output=discriminator(x)
  dcgan = Model(inputs=[gan_input], outputs=[gan_output])
  dcgan.compile(loss='binary_crossentropy', optimizer='adam')
  dcgan.name='dcgan'
  return dcgan

gan=create_dcgan(d,g)
gan.summary()

Model: "dcgan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128)               0         
_________________________________________________________________
generador (Sequential)       (None, 20, 60, 1)         619265    
_________________________________________________________________
discriminador (Sequential)   (None, 1)                 42465     
Total params: 661,730
Trainable params: 619,265
Non-trainable params: 42,465
_________________________________________________________________


## Cuasi Autoencoder que avanza paso a paso por el camino y genera las visuales según avanza

---

Esta red está compuesta de 2 subredes. La primera es una convolucional que permite comprimir todas las features en un array para concatenarlo con el array de dirección. La segunda es una red deconvolucional que a partir del vector comprimido concaatenado con las direcciones genera una nueva visual. El funcionamiento es muy similar a un autoencoder con el matiz de concatenar con las direcciones. Para aprovechar el aprendizaje adquirido en la GAN anterior se puede hacer transfer learning copiando muchas de sus capas.



### Encoder

In [56]:
# input matriz (20,60)
# output array 125
def create_encoder():
  
    encoder=Sequential()

    encoder.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),input_shape=(20,60,1),name='e_conv2d_1'))
    encoder.add(LeakyReLU(alpha=0.2))
    encoder.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='e_maxpool_1'))
    
    encoder.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),name='e_conv2d_2'))
    encoder.add(LeakyReLU(alpha=0.2))
    encoder.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1),name='e_maxpool_2'))
    
    encoder.add(Flatten())
    encoder.add(Dense(units=125, activation='relu'))
    
    encoder.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    encoder.name='encoder'
    return encoder
e = create_encoder()
e.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
e_conv2d_1 (Conv2D)          (None, 16, 56, 32)        832       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 56, 32)        0         
_________________________________________________________________
e_maxpool_1 (MaxPooling2D)   (None, 15, 55, 32)        0         
_________________________________________________________________
e_conv2d_2 (Conv2D)          (None, 11, 51, 32)        25632     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 11, 51, 32)        0         
_________________________________________________________________
e_maxpool_2 (MaxPooling2D)   (None, 10, 50, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 16000)             0   

### Decoder

In [48]:
# input array 128
# output matriz (20,60)

f = create_generator()
f.summary()

Model: "generador"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3200)              412800    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
conv2dt_1 (Conv2DTranspose)  (None, 10, 15, 64)        204864    
_________________________________________________________________
conv2dt_2 (Conv2DTranspose)  (None, 20, 60, 1)         1601      
Total params: 619,265
Trainable params: 619,265
Non-trainable params: 0
_________________________________________________________________


### Cuasi Autoencoder

In [61]:
def c_autoencoder(encoder, decoder):

  c_autoencoder_input = Input(shape=(20,60,1))
  x = encoder(c_autoencoder_input)
  p = Input(shape=(3,))
  x = keras.layers.Concatenate()([x,p])
  c_autoencoder_output=decoder(x)
  autoencoder = Model(inputs=[c_autoencoder_input, p], outputs=c_autoencoder_output)
  autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
  autoencoder.name='cuasi autoencoder'
  return autoencoder

autoencoder=c_autoencoder(e,f)
autoencoder.summary()

Model: "cuasi autoencoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 20, 60, 1)    0                                            
__________________________________________________________________________________________________
encoder (Sequential)            (None, 125)          2026589     input_24[0][0]                   
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 128)          0           encoder[3][0]                    
                                                                 input_25[0][0]   

# Entrenamiento

---



#Modelo generado final

---

In [0]:
# input matriz direcciones (n,3)
# output matriz visuales (n,20,60)

def generar_edificio(direcciones):

  return visuales


# Postprocesado de datos de salida

---

